<a href="https://colab.research.google.com/github/Otter2022/RowdyDatathon24/blob/main/pysparkanalysisofjourneynorth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beutifulsoup4
!pip install requests
!pip install pandas matplotlib seaborn cartopy
!pip install ffmpeg
!pip install pyspark


ERROR: Could not find a version that satisfies the requirement beutifulsoup4 (from versions: none)
ERROR: No matching distribution found for beutifulsoup4


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import concurrent.futures
import warnings
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType, StructType, StructField
import pandas as pd
import requests
from bs4 import BeautifulSoup
import concurrent.futures

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Monarch Butterfly Sightings") \
    .getOrCreate()

warnings.filterwarnings("ignore")

In [3]:


# Define the function to process the data for each year
def process_year(cur_type, year):
    fall_URL = f"https://journeynorth.org/sightings/querylist.html?map={cur_type}-fall&year={year}&season=fall"
    spring_URL = f"https://journeynorth.org/sightings/querylist.html?map={cur_type}-spring&year={year}&season=spring"

    print(f"Fetching fall URL: {fall_URL}")
    print(f"Fetching spring URL: {spring_URL}")

    fall_request = requests.get(fall_URL)
    spring_request = requests.get(spring_URL)

    # Parse HTML data with BeautifulSoup
    fall_soup = BeautifulSoup(fall_request.text, "html.parser")
    spring_soup = BeautifulSoup(spring_request.text, "html.parser")

    fall_table = fall_soup.find("table")
    spring_table = spring_soup.find("table")

    # Extract the tables into pandas dataframes
    fall_df = pd.read_html(str(fall_table))[0] if fall_table else pd.DataFrame()
    spring_df = pd.read_html(str(spring_table))[0] if spring_table else pd.DataFrame()

    return (year, fall_df), (year, spring_df)

# Function to scrape multiple years of data concurrently
def web_scrape_sighting_table(cur_type):
    fall_years = []
    spring_years = []
    years = list(range(1997, 2025))

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_year, cur_type, year) for year in years]
        for future in concurrent.futures.as_completed(futures):
            fall_result, spring_result = future.result()
            fall_years.append(fall_result)
            spring_years.append(spring_result)

    return fall_years, spring_years

# Types of monarch sightings to scrape
types = ["monarch-egg", "monarch-other", "monarch-adult", "monarch-roost", "monarch-larva", "monarch-captive"]

# Initialize lists for fall and spring sightings
df_fall_pairs = []
df_spring_pairs = []

# Iterate over all types and scrape data
for cur_type in types:
    fall_data, spring_data = web_scrape_sighting_table(cur_type)
    df_fall_pairs.append(fall_data)
    df_spring_pairs.append(spring_data)

# Function to convert pandas dataframes into PySpark DataFrames and save them
def process_and_save_data(fall_data, spring_data, cur_type):
    # Flatten fall and spring data
    fall_dfs = [x[1] for x in fall_data if not x[1].empty]
    spring_dfs = [x[1] for x in spring_data if not x[1].empty]

    if fall_dfs:
        fall_combined = pd.concat(fall_dfs, ignore_index=True)
        fall_spark_df = spark.createDataFrame(fall_combined)
        # Save or process further with PySpark
        fall_spark_df.write.mode("overwrite").parquet(f"sightings_{cur_type}_fall.parquet")

    if spring_dfs:
        spring_combined = pd.concat(spring_dfs, ignore_index=True)
        spring_spark_df = spark.createDataFrame(spring_combined)
        # Save or process further with PySpark
        spring_spark_df.write.mode("overwrite").parquet(f"sightings_{cur_type}_spring.parquet")

# Process and save all data for each type
for i, cur_type in enumerate(types):
    process_and_save_data(df_fall_pairs[i], df_spring_pairs[i], cur_type)


Fetching fall URL: https://journeynorth.org/sightings/querylist.html?map=monarch-egg-fall&year=1997&season=fall
Fetching spring URL: https://journeynorth.org/sightings/querylist.html?map=monarch-egg-spring&year=1997&season=spring
Fetching fall URL: https://journeynorth.org/sightings/querylist.html?map=monarch-egg-fall&year=1998&season=fall
Fetching spring URL: https://journeynorth.org/sightings/querylist.html?map=monarch-egg-spring&year=1998&season=spring
Fetching fall URL: https://journeynorth.org/sightings/querylist.html?map=monarch-egg-fall&year=1999&season=fall
Fetching spring URL: https://journeynorth.org/sightings/querylist.html?map=monarch-egg-spring&year=1999&season=spring
Fetching fall URL: https://journeynorth.org/sightings/querylist.html?map=monarch-egg-fall&year=2000&season=fall
Fetching spring URL: https://journeynorth.org/sightings/querylist.html?map=monarch-egg-spring&year=2000&season=spring
Fetching fall URL: https://journeynorth.org/sightings/querylist.html?map=monarch

In [4]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors

# Load the saved parquet files
fall_df = spark.read.parquet("sightings_monarch-adult_fall.parquet")
spring_df = spark.read.parquet("sightings_monarch-adult_spring.parquet")

from pyspark.sql.functions import isnan, when, count, col

# Filter out rows with NaN values in any of the relevant columns
fall_df = fall_df.filter(~isnan("Latitude") & ~isnan("Longitude") & ~isnan("Number"))
spring_df = spring_df.filter(~isnan("Latitude") & ~isnan("Longitude") & ~isnan("Number"))

# Select numeric columns for correlation analysis
# Changed 'Sightings' to 'Number' to match the actual column name
assembler = VectorAssembler(inputCols=["Latitude", "Longitude", "Number"], outputCol="features")

# Transform data into feature vectors
fall_features = assembler.transform(fall_df)
spring_features = assembler.transform(spring_df)

# Compute correlation matrix
fall_correlation_matrix = Correlation.corr(fall_features, "features").head()[0]
spring_correlation_matrix = Correlation.corr(spring_features, "features").head()[0]

print("Fall Correlation Matrix:\n", fall_correlation_matrix)
print("Spring Correlation Matrix:\n", spring_correlation_matrix)


Fall Correlation Matrix:
 DenseMatrix([[ 1.        ,  0.55462816, -0.00113485],
             [ 0.55462816,  1.        , -0.00245514],
             [-0.00113485, -0.00245514,  1.        ]])
Spring Correlation Matrix:
 DenseMatrix([[ 1.        ,  0.21614624, -0.02229468],
             [ 0.21614624,  1.        , -0.05165304],
             [-0.02229468, -0.05165304,  1.        ]])


In [5]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

# Assuming your dataset includes latitude, longitude, and number as potential predictors
# You can replace these with other relevant columns from your dataset
assembler = VectorAssembler(inputCols=["Latitude", "Longitude", "Number"], outputCol="features")
df_with_features = assembler.transform(fall_df)  # Apply for spring_df similarly

# Create the linear regression model
# Assuming 'Number' is the target variable you want to predict
lr = LinearRegression(featuresCol="features", labelCol="Number")

# Fit the model
lr_model = lr.fit(df_with_features)

# Print the coefficients and intercept
print(f"Coefficients: {lr_model.coefficients}")
print(f"Intercept: {lr_model.intercept}")

# Summarize the model over the training set
training_summary = lr_model.summary
print(f"RMSE: {training_summary.rootMeanSquaredError}")
print(f"R2: {training_summary.r2}")

Coefficients: [-1.5546047556888142e-13,-1.0400928589788479e-13,0.9999999999999999]
Intercept: 0.0
RMSE: 6.743706530941293e-10
R2: 1.0


In [6]:
from pyspark.ml.clustering import KMeans

# Apply k-means clustering to group regions by sightings and environmental factors
kmeans = KMeans(k=3, seed=1)  # Assuming you want 3 clusters
kmeans_model = kmeans.fit(df_with_features)

# Evaluate clustering results
clusters = kmeans_model.transform(df_with_features)
clusters.show()

# You can analyze the cluster centers to understand where the issues are occurring
print("Cluster Centers: ")
for center in kmeans_model.clusterCenters():
    print(center)


+----------+--------+---------------+--------------+--------+---------+------+-----+-----------------+----------+
|Unnamed: 0|    Date|           Town|State/Province|Latitude|Longitude|Number|Image|         features|prediction|
+----------+--------+---------------+--------------+--------+---------+------+-----+-----------------+----------+
|         1|12/27/01|        Orlando|            FL|    28.5|    -81.4|   1.0|  NaN| [28.5,-81.4,1.0]|         0|
|         2|12/21/01|    New Orleans|            LA|    30.0|    -90.1|   1.0|  NaN| [30.0,-90.1,1.0]|         0|
|         3|10/14/01|  New Baltimore|            MI|    42.7|    -82.8|   1.0|  NaN| [42.7,-82.8,1.0]|         0|
|         4|09/12/01|      Iowa City|            IA|    41.7|    -91.5|  15.0|  NaN|[41.7,-91.5,15.0]|         0|
|         5|09/11/01|      Haverhill|            MA|    42.8|    -71.1|   1.0|  NaN| [42.8,-71.1,1.0]|         0|
|         6|08/25/01|         Ottawa|            ON|    45.5|    -75.5|   1.0|  NaN| [45

In [9]:
from pyspark.sql import functions as F
from pyspark.sql.functions import year, to_date

# # Extract year from the 'Date' column and create a new 'Year' column
# fall_df = fall_df.withColumn("Year", F.year(F.to_date(fall_df.Date, "MM/dd/yy")))
fall_df = fall_df.withColumn("Year", F.year(F.to_date(fall_df.Date, "MM/dd/yy")))

# Filter out rows with years 2097, 2098, and 2099 in fall_df
fall_df = fall_df.filter(~year(to_date(fall_df.Date, "MM/dd/yy")).isin([2097, 2098, 2099]))


# Group data by year to perform time series analysis
fall_time_series_df = fall_df.groupBy("Year").agg(F.avg("Number").alias("Avg_Sightings"))

# Display trends over time
fall_time_series_df.orderBy("Year").show(30)


# # Extract year from the 'Date' column and create a new 'Year' column
spring_df = spring_df.withColumn("Year", F.year(F.to_date(spring_df.Date, "MM/dd/yy")))

# Filter out rows with years 2097, 2098, and 2099 in spring_df
spring_df = spring_df.filter(~year(to_date(spring_df.Date, "MM/dd/yy")).isin([2097, 2098, 2099]))

# Group data by year to perform time series analysis
fall_time_series_df = spring_df.groupBy("Year").agg(F.avg("Number").alias("Avg_Sightings"))

# Display trends over time
fall_time_series_df.orderBy("Year").show(30)


+----+------------------+
|Year|     Avg_Sightings|
+----+------------------+
|2000|               1.0|
|2001|3.3333333333333335|
|2002|3.3333333333333335|
|2003|               4.0|
|2004|               1.0|
|2006|               1.0|
|2007|               1.5|
|2008|               2.5|
|2009|               1.0|
|2010|1.0804455445544554|
|2011| 6.253463927376971|
|2012|  9.54109178164367|
|2013|3.0756079942210452|
|2014| 4.012957046503373|
|2015| 7.395120174799708|
|2016|3.0421901528013584|
|2017|3.5409392458100557|
|2018| 5.184031413612566|
|2019| 7.542740210213004|
|2020|3.8216817644742025|
|2021| 5.629227733934611|
|2022| 2.479225105189341|
|2023|246556.38794489094|
|2024| 2.890527509926262|
+----+------------------+

+----+------------------+
|Year|     Avg_Sightings|
+----+------------------+
|2001|            1012.0|
|2002|               1.0|
|2003|               1.1|
|2004|               1.0|
|2005|               1.0|
|2006|               1.0|
|2007|               1.0|
|2008|1.020

In [10]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.sql.functions import when

# Create a new column 'DeclineRisk' based on your logic (example)
# This example assumes a decline risk if the 'Number' of sightings is below a threshold
# You should replace this with your specific logic for determining decline risk
df_with_features = df_with_features.withColumn("DeclineRisk", when(df_with_features["Number"] < 10, 1).otherwise(0))

# Train a decision tree classifier on the dataset
dt = DecisionTreeClassifier(featuresCol="features", labelCol="DeclineRisk")
dt_model = dt.fit(df_with_features)

# Predict regions or time periods at risk of decline
predictions = dt_model.transform(df_with_features)
predictions.select("prediction", "features").show()

# You can evaluate the model's performance

+----------+-----------------+
|prediction|         features|
+----------+-----------------+
|       1.0| [28.5,-81.4,1.0]|
|       1.0| [30.0,-90.1,1.0]|
|       1.0| [42.7,-82.8,1.0]|
|       0.0|[41.7,-91.5,15.0]|
|       1.0| [42.8,-71.1,1.0]|
|       1.0| [45.5,-75.5,1.0]|
|       1.0|[31.3,-100.3,2.0]|
|       1.0| [31.8,-87.1,6.0]|
|       1.0| [40.9,-74.0,2.0]|
|       1.0| [28.5,-96.7,1.0]|
|       1.0| [28.6,-82.4,1.0]|
|       1.0| [34.3,-88.7,1.0]|
|       1.0| [42.8,-76.0,1.0]|
|       1.0| [40.8,-95.6,1.0]|
|       1.0| [42.8,-77.0,1.0]|
|       1.0| [42.8,-76.0,1.0]|
|       1.0| [41.4,-74.2,1.0]|
|       1.0| [41.4,-74.2,1.0]|
|       1.0| [44.9,-93.3,1.0]|
|       1.0|[48.0,-102.1,1.0]|
+----------+-----------------+
only showing top 20 rows



In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
!pip install geopy
!pip install sqlalchemy_mate==2.0.0.0
!pip install uszipcode==1.0.1
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from geopy.geocoders import Nominatim


In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from geopy.geocoders import Nominatim
from uszipcode import SearchEngine

def get_county_code(latitude, longitude):
    """
    Gets the county FIPS code based on latitude and longitude coordinates.
    """
    search = SearchEngine(simple_zipcode=True)  # Initialize SearchEngine with simple_zipcode=True for faster lookups
    result = search.by_coordinates(latitude, longitude, radius=10, returns=1)
    if result:
        return result[0].county
    else:
        return None  # Return None if no county is found

# Create a user-defined function (UDF) for Spark to use
get_county_code_udf = udf(get_county_code, StringType())

# Assuming you have a Spark DataFrame called 'df' with 'latitude' and 'longitude' columns
# Add a new column 'county_code' with the county FIPS code
df = df.withColumn("county_code", get_county_code_udf(df["latitude"], df["longitude"]))

AttributeError: module 'sqlalchemy_mate' has no attribute 'ExtendedBase'

In [21]:
fall_output_path = "/content/drive/MyDrive/Datathon24/fall_monarch_butterfly_sightings.parquet"
fall_df.write.mode("overwrite").parquet(fall_output_path)
fall_df.write.mode("overwrite").csv("/content/drive/MyDrive/Datathon24/fall_monarch_butterfly_sightings.csv")


df = spark.read.parquet("/content/drive/MyDrive/Datathon24/fall_monarch_butterfly_sightings.parquet")
df.show()


spring_output_path = "/content/drive/MyDrive/Datathon24/spring_monarch_butterfly_sightings.parquet"
spring_df.write.mode("overwrite").parquet(spring_output_path)
spring_df.write.mode("overwrite").csv("/content/drive/MyDrive/Datathon24/spring_monarch_butterfly_sightings.csv")

df = spark.read.parquet("/content/drive/MyDrive/Datathon24/spring_monarch_butterfly_sightings.parquet")
df.show()


+----------+--------+---------------+--------------+--------+---------+------+-----+----+
|Unnamed: 0|    Date|           Town|State/Province|Latitude|Longitude|Number|Image|Year|
+----------+--------+---------------+--------------+--------+---------+------+-----+----+
|         1|12/27/01|        Orlando|            FL|    28.5|    -81.4|   1.0|  NaN|2001|
|         2|12/21/01|    New Orleans|            LA|    30.0|    -90.1|   1.0|  NaN|2001|
|         3|10/14/01|  New Baltimore|            MI|    42.7|    -82.8|   1.0|  NaN|2001|
|         4|09/12/01|      Iowa City|            IA|    41.7|    -91.5|  15.0|  NaN|2001|
|         5|09/11/01|      Haverhill|            MA|    42.8|    -71.1|   1.0|  NaN|2001|
|         6|08/25/01|         Ottawa|            ON|    45.5|    -75.5|   1.0|  NaN|2001|
|         1|11/09/02|     San Angelo|            TX|    31.3|   -100.3|   2.0|  NaN|2002|
|         2|09/30/02|       Beatrice|            AL|    31.8|    -87.1|   6.0|  NaN|2002|
|         